In [1]:
import torch
import seaborn as sns
import sys
from pathlib import Path
sys.path.insert(0, "../..")

In [2]:
from src.data import make_dataset
from src.models import imagemodels
from src.models import train_model
import gin


2023-04-30 20:03:31.075880: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-30 20:03:31.926318: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
gin.parse_config_file("model.gin")

ParsedConfigFileIncludesAndImports(filename='model.gin', imports=['gin.torch.external_configurables'], includes=[])

We will be using `gin-config` to easily keep track of our experiments, and to easily save the different things we did during our experiments.

The `model.gin` file is a simple file that will try to load parameters for funcitons that are already imported. 

So, if you wouldnt have imported train_model, the ginfile would not be able to parse settings for train_model.trainloop and will give an error.

We can print all the settings that are operational with `gin.operative_config_str()` once we have loaded the functions to memory.

So, while `.get_MNIST()` has two parameters that need to be set (a batchsize and a datadir), we can now load the function without having to do that: gin has done it already for us.

In [4]:
train_dataloader, test_dataloader = make_dataset.get_MNIST()

In [5]:
print(gin.config_str())

import gin.torch.external_configurables

# Parameters for get_MNIST:
# ==============================================================================
get_MNIST.batch_size = 32
get_MNIST.data_dir = '../../data/raw'

# Parameters for NeuralNetwork:
# ==============================================================================
NeuralNetwork.num_classes = 10
NeuralNetwork.units1 = 512

# Parameters for trainloop:
# ==============================================================================
trainloop.epochs = 10
trainloop.learning_rate = 0.001
trainloop.log_dir = '../../models/gtest/'
trainloop.loss_fn = @CrossEntropyLoss()
trainloop.optimizer = @Adam



A big advantage is that we can save this config as a file; that way it is easy to track what you changed during your experiments.

In [6]:
from src.models import metrics
accuracy = metrics.Accuracy()

In [7]:
import torch.optim as optim
gin.parse_config_file("model.gin")

units = [256, 128, 64]
learning_rate = [0.01, 0.001, 0.0001]

for unit in units:
    for lr in learning_rate:
        gin.bind_parameter("NeuralNetwork.units2", unit)
        gin.bind_parameter("trainloop.learning_rate", lr)
    
        model = imagemodels.NeuralNetwork()

        model, test_loss =  train_model.trainloop(
            epochs=3,
            model=model,
            metrics=[accuracy],
            train_dataloader=train_dataloader,
            test_dataloader=test_dataloader,
            train_steps=len(train_dataloader),
            eval_steps=len(test_dataloader),
            tunewriter=["tensorboard", "gin"]
        )

2023-04-30 20:03:34.630 | INFO     | src.data.data_tools:dir_add_timestamp:129 - Logging to ../../models/gtest/20230430-2003
100%|██████████| 1875/1875 [00:32<00:00, 57.46it/s]
2023-04-30 20:04:09.879 | INFO     | src.models.train_model:trainloop:180 - Epoch 0 train 0.5398 test 0.4946 metric ['0.8313']
100%|██████████| 1875/1875 [00:40<00:00, 46.79it/s]
2023-04-30 20:04:52.753 | INFO     | src.models.train_model:trainloop:180 - Epoch 1 train 0.4477 test 0.4607 metric ['0.8372']
100%|██████████| 1875/1875 [00:50<00:00, 36.77it/s]
2023-04-30 20:05:46.769 | INFO     | src.models.train_model:trainloop:180 - Epoch 2 train 0.4250 test 0.4833 metric ['0.8298']
100%|██████████| 3/3 [02:12<00:00, 44.04s/it]
2023-04-30 20:05:46.777 | INFO     | src.data.data_tools:dir_add_timestamp:129 - Logging to ../../models/gtest/20230430-2005
100%|██████████| 1875/1875 [00:30<00:00, 62.50it/s]
2023-04-30 20:06:18.513 | INFO     | src.models.train_model:trainloop:180 - Epoch 0 train 0.4805 test 0.4488 metric

Run the experiment, and study the result with tensorboard. 

Locally, it is easy to do that with VS code itself. On the server, you have to take these steps:

- in the terminal, navigate to ~/code/ML22 
- activate the python environment for the shell with `poetry shell`. Note how the correct environment is being activated.
- run `tensorboard --logdir=models` in the terminal
- tensorboard will launch at `localhost:6006` and vscode will notify you that the port is forwarded
- you can either press the `launch` button in VScode or open your local browser at `localhost:6006`



Experiment with things like:

- changing the amount of units1 and units2 to values between 16 and 1024. Use factors of 2: 16, 32, 64, etc.
- changing the batchsize to values between 4 and 128. Again, use factors of two.
- all your experiments are saved in the `models` directory, with a timestamp. Inside you find a saved_config.gin file, that 
contains all the settings for that experiment. The `events` file is what tensorboard will show.
- plot the result in a heatmap: units vs batchsize.
- changing the learningrate to values between 1e-2 and 1e-5 
- changing the optimizer from SGD to one of the other available algoritms at [torch](https://pytorch.org/docs/stable/optim.html) (scroll down for the algorithms)

A note on train_steps: this is a setting that determines how often you get an update. 
Because our complete dataset is 938 (60000 / 64) batches long, you will need 938 trainstep to cover the complete 60.000 images.

This can actually be a bit confusion, because every value below 938 changes the meaning of `epoch` slightly, because one epoch is no longer
the full dataset, but simply `trainstep` batches. Setting trainsteps to 100 means you need to wait twice as long before you get feedback on the performance,
as compared to trainsteps=50. You will also see that settings trainsteps to 100 improves the learning, but that is simply because the model has seen twice as 
much examples as compared to trainsteps=50.

This implies that it is not usefull to compare trainsteps=50 and trainsteps=100, because setting it to 100 will always be better.
Just pick an amount, and adjust your number of epochs accordingly.